<a href="https://colab.research.google.com/github/sundarjhu/EscueladeVerano_2021/blob/main/Escuela_de_Verano_2021_Demo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Accessing data from SIMBAD, demonstrating ADQL basics**
---



---



In [1]:
from astropy.table import Table, join
import numpy as np
!pip install pyvo
import pyvo as vo

     |████████████████████████████████| 802 kB 3.6 MB/s 
  Created wheel for mimeparse: filename=mimeparse-0.1.3-py3-none-any.whl size=3864 sha256=27664e073e89a2c2cf15546d4ae3747a06baf4af8a407ac82c6bafe54b67cbf2
  Stored in directory: /root/.cache/pip/wheels/49/b4/2d/0081759ae1833bd694024801f7aacddcda8a687e8d5fbaeebd
Successfully built mimeparse


## Create a table of source identifiers, save to VOTable. This will serve as input for our queries.

In [2]:
id = ['lam Vel', 'IRAS 16205-2626', 'eta Car', 'alp Cen', 'IRC +10216'] #unique identifiers for three stars
hemisphere = ['Southern', 'Southern', 'Southern', 'Southern', 'Northern'] #which celestial hemisphere?
input_tab = Table([id, hemisphere], names = ('ID', 'hemisphere'))
input_tab.pprint()
input_tab.write('input_tab.vot', format = 'votable', overwrite = True)

       ID       hemisphere
--------------- ----------
        lam Vel   Southern
IRAS 16205-2626   Southern
        eta Car   Southern
        alp Cen   Southern
     IRC +10216   Northern


### Automatically obtain the url for the SIMBAD TAP server by doing a regsearch using PyVO

In [3]:
r = vo.regsearch(servicetype = 'tap', keywords = ['Simbad'])
SIMBAD_service = r[0].service

In [4]:
# This is the url to the TAP server
SIMBAD_service.baseurl

'http://simbad.u-strasbg.fr:80/simbad/sim-tap'

In [5]:
#Select all columns from the first row of basic data
query = """ SELECT top 1 * FROM basic """ # change "1" to any number to view that many rows in the output
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results.colnames
#You can use this to inspect the column names and their content in the table

['coo_bibcode',
 'coo_err_angle',
 'coo_err_maj',
 'coo_err_maj_prec',
 'coo_err_min',
 'coo_err_min_prec',
 'coo_qual',
 'coo_wavelength',
 'dec_prec',
 'galdim_angle',
 'galdim_bibcode',
 'galdim_majaxis',
 'galdim_majaxis_prec',
 'galdim_minaxis',
 'galdim_minaxis_prec',
 'galdim_qual',
 'hpx',
 'morph_bibcode',
 'morph_qual',
 'morph_type',
 'nbref',
 'oid',
 'otype',
 'plx_bibcode',
 'plx_err',
 'plx_err_prec',
 'plx_prec',
 'plx_qual',
 'plx_value',
 'pm_bibcode',
 'pm_err_angle',
 'pm_err_maj',
 'pm_err_maj_prec',
 'pm_err_min',
 'pm_err_min_prec',
 'pm_qual',
 'pmdec',
 'pmdec_prec',
 'pmra',
 'pmra_prec',
 'ra_prec',
 'rvz_bibcode',
 'rvz_err',
 'rvz_err_prec',
 'rvz_nature',
 'rvz_qual',
 'rvz_radvel',
 'rvz_radvel_prec',
 'rvz_redshift',
 'rvz_redshift_prec',
 'rvz_type',
 'sp_bibcode',
 'sp_qual',
 'sp_type',
 'update_date',
 'vlsr',
 'main_id',
 'otype_txt',
 'ra',
 'dec']

In [6]:
#How many rows are there in this table?
query = """ select count(coo_bibcode) as NUMROWS from basic"""
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results

numrows
int64
11898086


In [7]:
#HEY, waitaminit! I asked for the column to be in all caps, why isn't it?
#Because ADQL is case-insensitive unless you enclose the argument in quotes
query = """ select count(coo_bibcode) as "NUMROWS" from basic"""
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results

NUMROWS
int64
11898086


In [8]:
#SUBSETS with WHERE: How many rows are there in this table with DEC < 0?
query = """ select count(coo_bibcode) as NUMROWS from basic WHERE dec < 0"""
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results

query = """ select top 100 coo_bibcode as NUMROWS from basic WHERE dec < 0"""
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results


numrows
object
2013yCat.2319....0L
2018yCat.1345....0G
2018ATel11731....1P
2013MNRAS.428.1281J
2018yCat.1345....0G
2018yCat.1345....0G
2013MNRAS.428.1281J
""
2018yCat.1345....0G


# There are three important tables in SIMBAD: `basic`, `ident`, and `ids`. 

# `basic` contains the basic information for each source (positions, velocities, photometry, redshift, etc.). If a source is known by many names, there is an entry for EACH such name recognized by SIMBAD.

# Each source in these tables is identified by a unique identifier, a long integer called *oid* (in `basic`) and *oidref* (in `ident`, `ids`). You can connect a source to its various names using these two columns.

# You can obtain a list of references for the sources in the same way using the tables `ref` and `has_ref`.

In [9]:
# Get some information from the basic table for the object with name 'CW Leo', which is NOT its standard SIMBAD name.
#
query = """
-- Basic data from an object given one of its identifiers.
SELECT basic.OID,
       RA,
       DEC,
       main_id AS "Main identifier",
       coo_bibcode AS "Coord Reference",
       nbref AS "NbReferences",
       plx_value as "Parallax",
       rvz_radvel as "Radial velocity",
       galdim_majaxis,
       galdim_minaxis,
       galdim_angle AS "Galaxy ellipse angle"
FROM basic JOIN ident ON oidref = oid
WHERE id = 'CW Leo'
"""
results = SIMBAD_service.search(query).to_table()
results

oid,ra,dec,Main identifier,Coord Reference,NbReferences,Parallax,Radial velocity,galdim_majaxis,galdim_minaxis,Galaxy ellipse angle
,deg,deg,,,,mas,km / s,arcmin,arcmin,deg
int64,float64,float64,object,object,int32,float64,float64,float32,float32,int16
1657507,146.989193,13.278768,IRC +10216,2003yCat.2246....0C,2200,10.79,--,--,--,--


### Get all the identifiers for one source

In [10]:
query = """select t0.main_id, t1.id, t2.ids from basic as t0, ident as t1, ids as t2 where t0.oid = t1.oidref and t1.id = 'CW Leo' and t1.oidref = t2.oidref"""
results = SIMBAD_service.search(query).to_table()
results

main_id,id,ids
object,object,object
IRC +10216,V* CW Leo,ATO J146.9891+13.2788|LEDA 1427054|TIC 258671744|CGCS 2619|IRAS 09452+1330|IRC +10216|PK 221+45 1|RAFGL 1381|SCM 50|V* CW Leo|Zel 0945+135|[KKH2001] 55|[LFO93] 0945+13|2MASS J09475740+1316435|JCMTSE J094757.1+131644|JCMTSF J094757.1+131644|SDSS J094757.42+131643.6|LINEAR 17154286|NAME Peanut Nebula|CRTS J094757.4+131643|PN G221.4+45.0


# How to upload a table!
### Get all the identifiers for a list of sources 

In [11]:
input_tab

ID,hemisphere
str15,str8
lam Vel,Southern
IRAS 16205-2626,Southern
eta Car,Southern
alp Cen,Southern
IRC +10216,Northern


In [12]:
query = """select mytable.id, t2.ids from mytable, ident as t1, ids as t2 where t1.id = mytable.ID and t1.oidref = t2.oidref"""
results = SIMBAD_service.search(query, uploads = {'mytable': 'input_tab.vot'}).to_table()
results

ID,ids
object,object
lam Vel,TIC 31975064|PLX 2187|* lam Vel|CCDM J09080-4326A|CD-42 4990|CPC 0 7024|CPD-42 3366|CSI-42 4990 21|CSV 6689|FK5 345|GC 12623|GCRV 5989|GEN# +1.00078647|GSC 07689-02617|HD 78647|HIC 44816|HIP 44816|HR 3634|IDS 09043-4302 A|JP11 1770|N30 2164|PPM 313999|SACS 200|SAO 220878|SKY# 17613|TD1 13594|TYC 7689-2617-1|UBV 8792|V* lam Vel|[CS62] E4 76|uvby98 100078647|2MASS J09075977-4325571|PLX 2187.00|WDS J09080-4326A|NAME Suhail|WEB 8567
IRAS 16205-2626,IRAS 16205-2626
eta Car,Gaia DR2 5350358580171706624|LLNS 2725|TIC 458859916|[ARV2008] 173|[KRL2007b] 93|* eta Car|* eta Car A|1A 1044-59|1H 1045-597|2E 2318|3A 1042-595|4U 1053-58|4U 1037-60|AAVSO 1041-59|ALS 1868|CD-59 3306|CEL 3689|CPC 20 3145|CPD-59 2620|CSI-59 2620 41|EM* MWC 214|GC 14799|GCRV 6693|GPS 1043-595|H 1044-59|HD 93308|HR 4210|Hen 3-481|IRAS 10431-5925|JP11 1994|LS 1868|MOST 1042-594|NOVA Car 1843|PPM 339408|RAFGL 4114|SAO 238429|SKY# 20584|V* eta Car|[AMM2003] 50|[S87b] IRS 23|2MASS J10450360-5941040|Cl Trumpler 16 183|JCMTSE J104503.7-594102|1E 104306-5925.2|1ES 1043-59.4|2E 1043.1-5925|CXOCyg J104503.58-594103.7|JCMTSF J104503.7-594102|XMMU J104503.8-594105|1FGL J1045.2-5942|0FGL J1045.6-5937|1AGL J1043-5936|PBC J1044.8-5942|2FGL J1045.0-5941|TYC 8626-2809-1|CSI-59-10431|GCRV 6692|* eta Car B|HD 93308B|2XMM J104503.4-594103|1AGLR J1044-5944|1AGL J1044-5937|1FHL J1045.0-5943|3FGL J1045.1-5941|SWIFT J1044.8-5941|WDS J10451-5941|WRAY 15-640|2FHL J1045.2-5942|UCAC4 152-053215|WEB 9578|3FHL J1045.1-5941|4FGL J1045.1-5940
alp Cen,* alf Cen|2E 3308|2RE J143941-605000|2RE J1439-605|CCDM J14396-6050AB|CPD-60 5483|CSI-60 5483 43|Ci 20 875|FK5 538|GC 19728|GCRV 8519|HD 128620J|IDS 14328-6025 AB|LPM 534|RE J1439-605|RE J143944-605008|SBC7 520|1E 143556-6037.3|1ES 1435-60.6|2E 1435.9-6037|2EUVE J1439-60.8|EUVE J1439-60.8|[TSA98] J143948.42-605021.66|WDS J14396-6050AB|RX J1439.5-6050|IRAS 14359-6037|SAO 252838|1RXS J143940.4-605020|PM 14362-6038|WDS J14403-6051AB|** RHD 1|1E 143555-6037.6|CD-60 5293|** LDS 494AB|uvby98 100128620|GES J14392972-6049560
IRC +10216,ATO J146.9891+13.2788|LEDA 1427054|TIC 258671744|CGCS 2619|IRAS 09452+1330|IRC +10216|PK 221+45 1|RAFGL 1381|SCM 50|V* CW Leo|Zel 0945+135|[KKH2001] 55|[LFO93] 0945+13|2MASS J09475740+1316435|JCMTSE J094757.1+131644|JCMTSF J094757.1+131644|SDSS J094757.42+131643.6|LINEAR 17154286|NAME Peanut Nebula|CRTS J094757.4+131643|PN G221.4+45.0


# Simple cone search
>## In this example, we select the location RA = 10 deg, DEC = 5 deg, and search for SIMBAD matches within a 1 deg radius of it.
>## We arrange the results in ascending order of angular distance using the DISTANCE function.

In [82]:
query = """
-- Display basic data about objects contained in a given circle and whose mag B < 9.0.
SELECT basic.OID,
       RA,
       DEC,
       DISTANCE(POINT('ICRS', RA, DEC), POINT('ICRS', 10, 5)) * 60 as dist_arcmin,
       main_id AS "Main identifier",
       coo_bibcode AS "BiblioReference",
       nbref AS "NbReferences",
       plx_value as "Parallax",
       rvz_radvel as "Radial velocity",
       galdim_majaxis,
       galdim_minaxis,
       galdim_angle AS "Galaxy ellipse angle"
FROM basic JOIN flux ON oidref = oid
WHERE filter = 'B'
      AND flux < 9.0
      AND CONTAINS(POINT('ICRS', RA, DEC), CIRCLE('ICRS', 10, 5, 1)) = 1
ORDER BY dist_arcmin
"""
results = SIMBAD_service.search(query).to_table()
results

oid,ra,dec,dist_arcmin,Main identifier,BiblioReference,NbReferences,Parallax,Radial velocity,galdim_majaxis,galdim_minaxis,Galaxy ellipse angle
,deg,deg,,,,,mas,km / s,arcmin,arcmin,deg
int64,float64,float64,float64,object,object,int32,float64,float64,float32,float32,int16
1404715,10.2080007087009,4.4783143611205,33.681574649245,HD 3819,2018yCat.1345....0G,12,4.2588,-3.05,--,--,--
1404698,9.2379259243896,5.4125099414372,51.8275257870488,HD 3398,2018yCat.1345....0G,21,9.1357,1.61,--,--,--
1414548,10.6367082889976,5.6753557257451,55.57664316161532,HD 3992,2018yCat.1345....0G,9,2.9468,--,--,--,--


# Cone search for list of sources in an input table
>## In this example, we select the locations (RA, DEC) specified for a list of sources in an input table and search for SIMBAD matches within a 1 arcmin radius of each position.
>## For each source in the input table, the matches are arranged in ascending order of angular separation using the DISTANCE function. The ORDER BY clause is used to first arrange the results in order of the unique identifier, then (for each unique identifer) by increasing match distance.

In [83]:
id = [1000001, 1000012, 1005016, 1052200] #some unique long-integer identifier
RA = [74.2, 159.4, 55.5, 355.959]
DEC = [44.6, -13.39, 62.7, 3.209]
input_table = Table([id, RA, DEC], names = ('ID', 'RA', 'DEC'))
input_table.write('input_tab.vot', format = 'votable', overwrite = True)

#Notice how the match radius is no longer hard-coded. We can change the radius outside
#   of the query string as follows. The query string changes in an automated fashion.
#   This is useful if we need to change the match radius outside the code block
#   that contains the query. It can then be fed as an argument to the query block.
match_radius = '2.0 / 60' #the match radius is set to 2 arc minutes.

query = """
SELECT a.ID, a.RA, a.DEC, main_id as "Main identifier", 
  b.RA AS matched_RA, b.DEC AS matched_DEC, coo_bibcode AS "Coord_BibRef",
  DISTANCE(POINT('ICRS', b.RA, b.DEC), POINT('ICRS', a.RA, a.DEC)) * 60 AS dist_arcmin,
  nbref AS "NbReferences", plx_value AS "Parallax",
  rvz_radvel AS "Radial velocity"
FROM mytable a LEFT JOIN basic b
ON CONTAINS(POINT('ICRS', b.RA, b.DEC), CIRCLE('ICRS', a.RA, a.DEC, """ + match_radius + """)) = 1
ORDER BY ID, dist_arcmin
"""

results = SIMBAD_service.search(query, uploads = {'mytable': 'input_tab.vot'}).to_table()
results

ID,RA,DEC,Main identifier,matched_ra,matched_dec,Coord_BibRef,dist_arcmin,NbReferences,Parallax,Radial velocity
,,,,deg,deg,,,,mas,km / s
int64,float64,float64,object,float64,float64,object,float64,int32,float64,float64
1000001,74.2,44.6,2MASX J04565269+4435157,74.21954166666667,44.587702777777785,2006AJ....131.1163S,1.1142355907175892,6,--,10444.549540259291
1000012,159.4,-13.39,V* U Hya,159.38863730625002,-13.384542478611111,2007A&A...474..653V,0.7396666784041439,289,4.8,-25.8
1005016,55.5,62.7,,--,--,,--,--,--,--
1052200,355.959,3.209,LSPM J2343+0313,355.9472432949003,3.2295778966234,2018yCat.1345....0G,1.421422719250529,1,3.1122,--


# Cone search for list of sources using a virtual input table
>## This example uses the same data as the previous one; the only difference is that it avoids writing to (or reading from) a file, and doesn't require an actual file to be passed as input to the query. Instead, we use a byte stream through the BytesIO method.

In [92]:
id = [1000001, 1000012, 1005016, 1052200] #some unique long-integer identifier
RA = [74.2, 159.4, 55.5, 355.959]
DEC = [44.6, -13.39, 62.7, 3.209]
input_table = Table([id, RA, DEC], names = ('ID', 'RA', 'DEC'))

#This is the only part of the code above that changes
from io import BytesIO
from astropy.io import votable
votable_object = BytesIO()
votable.writeto(votable.from_table(input_table), votable_object)
votable_object.seek(0)
#The rest of the code remains the same, except that we feed as input
# to the query not the VOTable file, but the VOTable object.

match_radius = str(2.0 / 60) #the match radius is set to 2 arc minutes.

query = """
SELECT a.ID, a.RA, a.DEC, main_id as "Main identifier", 
  b.RA AS matched_RA, b.DEC AS matched_DEC, coo_bibcode AS "Coord_BibRef",
  DISTANCE(POINT('ICRS', b.RA, b.DEC), POINT('ICRS', a.RA, a.DEC)) * 60 AS dist_arcmin,
  nbref AS "NbReferences", plx_value AS "Parallax",
  rvz_radvel AS "Radial velocity"
FROM mytable a LEFT JOIN basic b
ON CONTAINS(POINT('ICRS', b.RA, b.DEC), CIRCLE('ICRS', a.RA, a.DEC, """ + match_radius + """)) = 1
ORDER BY ID, dist_arcmin
"""

#The following line has changed
results = SIMBAD_service.search(query, uploads = {'mytable': votable_object}).to_table()
results

ID,RA,DEC,Main identifier,matched_ra,matched_dec,Coord_BibRef,dist_arcmin,NbReferences,Parallax,Radial velocity
,,,,deg,deg,,,,mas,km / s
int64,float64,float64,object,float64,float64,object,float64,int32,float64,float64
1000001,74.2,44.6,2MASX J04565269+4435157,74.21954166666667,44.587702777777785,2006AJ....131.1163S,1.1142355907175892,6,--,10444.549540259291
1000012,159.4,-13.39,V* U Hya,159.38863730625002,-13.384542478611111,2007A&A...474..653V,0.7396666784041439,289,4.8,-25.8
1005016,55.5,62.7,,--,--,,--,--,--,--
1052200,355.959,3.209,LSPM J2343+0313,355.9472432949003,3.2295778966234,2018yCat.1345....0G,1.421422719250529,1,3.1122,--


>## The above method is useful when we're dealing with large tables and/or we want to manipulate the table columns before feeding it to the search, without affecting the original table.

In [104]:
id = [1000001, 1000012, 1005016, 1052200] #some unique long-integer identifier
RA = [74.2, 159.4, 55.5, 355.959]
DEC = [44.6, -13.39, 62.7, 3.209]

#Some extra columns that make the table somewhat larger than before
Facility = ['Butter Glider', 'Magic Murder Bag', 'Vacuum Boom-Broom', 'The Revenge Society']
Saturated = [True, True, False, True]
ObsDate = ['2016', '2015', '1999', '2016']
Observer = ['Fitzcarraldo, Malcolm', 'Killinger, Henry', 'Venture, Thaddeus S.', 'Fântomas, Hamilton G.']
input_table = Table([id, RA, DEC, Facility, Saturated, ObsDate, Observer], \
                    names = ('ID', 'RA', 'DEC', 'Facility', 'Saturated', 'ObsDate', 'Observer'))

from io import BytesIO
from astropy.io import votable
votable_object = BytesIO()
#This line has changed. Note how only the necessary columns are being written to the VOTable object.
votable.writeto(votable.from_table(input_table['ID', 'RA', 'DEC']), votable_object)
votable_object.seek(0)

match_radius = str(5.0 / 60) #the match radius is set to 5 arc minutes.

query = """
SELECT a.ID, a.RA, a.DEC, main_id as "Main identifier", 
  b.RA AS matched_RA, b.DEC AS matched_DEC, coo_bibcode AS "Coord_BibRef",
  DISTANCE(POINT('ICRS', b.RA, b.DEC), POINT('ICRS', a.RA, a.DEC)) * 60 AS dist_arcmin,
  nbref AS "NbReferences", plx_value AS "Parallax",
  rvz_radvel AS "Radial velocity"
FROM mytable a LEFT JOIN basic b
ON CONTAINS(POINT('ICRS', b.RA, b.DEC), CIRCLE('ICRS', a.RA, a.DEC, """ + match_radius + """)) = 1
ORDER BY ID, dist_arcmin ASC
"""

results = SIMBAD_service.search(query, uploads = {'mytable': votable_object}).to_table()

#If needed, we can now add back the columns we ignored in the input table
"""IMPORTANT NOTE: the query results table MUST be the first argument to the join command in order to retain the ordering of distance.
  You can verify that this order isn't always preserved if input_table is used as the first argument by changing the line below.
"""
output_table = join(results, input_table, keys = ['ID', 'RA', 'DEC'], join_type = 'left')
output_table

ID,RA,DEC,Main identifier,matched_ra,matched_dec,Coord_BibRef,dist_arcmin,NbReferences,Parallax,Radial velocity,Facility,Saturated,ObsDate,Observer
,,,,deg,deg,,,,mas,km / s,,,,
int64,float64,float64,object,float64,float64,object,float64,int32,float64,float64,str19,bool,str4,str21
1000001,74.2,44.6,2MASX J04565269+4435157,74.21954166666667,44.587702777777785,2006AJ....131.1163S,1.1142355907175892,6,--,10444.549540259291,Butter Glider,True,2016,"Fitzcarraldo, Malcolm"
1000001,74.2,44.6,RAFGL 6319S,74.18033333333334,44.544888888888885,2003yCat.2246....0C,3.4118373762729273,28,--,--,Butter Glider,True,2016,"Fitzcarraldo, Malcolm"
1000001,74.2,44.6,BD+44 1060,74.2362658724653,44.6539405854728,2018yCat.1345....0G,3.5878574683719457,5,10.2859,9.0,Butter Glider,True,2016,"Fitzcarraldo, Malcolm"
1000001,74.2,44.6,ZOA J04570533+4433410,74.2722083,44.5613889,2016MNRAS.460..923R,3.8587007669985534,1,--,14506.400940488247,Butter Glider,True,2016,"Fitzcarraldo, Malcolm"
1000001,74.2,44.6,GSC 02906-00583,74.1741603053551,44.5356486149499,2018yCat.1345....0G,4.015959900184117,2,0.2718,--,Butter Glider,True,2016,"Fitzcarraldo, Malcolm"
1000001,74.2,44.6,ZOA J04562585+4438320,74.1077083,44.6422222,2016MNRAS.460..923R,4.685349626800239,1,--,10364.478381972509,Butter Glider,True,2016,"Fitzcarraldo, Malcolm"
1000012,159.4,-13.39,V* U Hya,159.38863730625002,-13.384542478611111,2007A&A...474..653V,0.7396666784041439,289,4.8,-25.8,Magic Murder Bag,True,2015,"Killinger, Henry"
1000012,159.4,-13.39,PMN J1037-1322,159.48458333333335,-13.390527777777777,1998AJ....115.1693C,4.937138897037928,1,--,--,Magic Murder Bag,True,2015,"Killinger, Henry"


## Selecting only the nearest-neighbour match for each source in the above example
> ### Since we've already arranged the matches in ascending order of distance, and performed the join using the query results as the first argument, we simply select the first entry for each source. This can be implemented using `numpy`'s `unique` method.

In [105]:
_, u = np.unique(output_table['ID'], return_index = True)
output_table[u]

ID,RA,DEC,Main identifier,matched_ra,matched_dec,Coord_BibRef,dist_arcmin,NbReferences,Parallax,Radial velocity,Facility,Saturated,ObsDate,Observer
,,,,deg,deg,,,,mas,km / s,,,,
int64,float64,float64,object,float64,float64,object,float64,int32,float64,float64,str19,bool,str4,str21
1000001,74.2,44.6,2MASX J04565269+4435157,74.21954166666667,44.587702777777785,2006AJ....131.1163S,1.1142355907175892,6,--,10444.549540259291,Butter Glider,True,2016,"Fitzcarraldo, Malcolm"
1000012,159.4,-13.39,V* U Hya,159.38863730625002,-13.384542478611111,2007A&A...474..653V,0.7396666784041439,289,4.8,-25.8,Magic Murder Bag,True,2015,"Killinger, Henry"
1005016,55.5,62.7,Gaia DR2 486856674972428928,55.57845437955,62.625958411863046,2018yCat.1345....0G,4.940510440439374,1,3.2331,--,Vacuum Boom-Broom,False,1999,"Venture, Thaddeus S."
1052200,355.959,3.209,LSPM J2343+0313,355.9472432949003,3.2295778966234,2018yCat.1345....0G,1.421422719250529,1,3.1122,--,The Revenge Society,True,2016,"Fântomas, Hamilton G."
